In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler  # for feature scaling
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve

In [ ]:
columns = [
'patient_id',
'avg_right_eigen',
'avg_sink_idx',
'avg_source_infl',
'avg_sink_conn',
'labels',
'channel']

Create a Pandas dataframe of these columns for the following model. We named it model_df

In [ ]:
groups = model_df['patient_id']
y_test_pred_optimal_all = np.zeros((int(model_df.shape[0]),3))
y_test_all = np.zeros((int(model_df.shape[0]),3))
y_test_pred_all = np.zeros((int(model_df.shape[0]),3))

num_iterations = len(patients) #insert list of engel 1 patients here
optimal_thresholds_all = np.zeros((num_iterations,3))
num_classes = 2

sss = LeaveOneGroupOut() #Leave One Out Cross Validation

for fold_idx, (train_index, test_index) in enumerate(sss.split(x, y, groups)):

    test_patient_ids = np.unique(groups[test_index])

    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    train_data = pd.DataFrame(x_train)
    train_data['labels'] = y_train
    train_data_oversampled = train_data.groupby(groups[train_index]).apply(oversample_group).reset_index(drop=True) #oversample training data
    
    x_train_resampled = train_data_oversampled.drop(columns='labels').values
    y_train_resampled = train_data_oversampled['labels'].values

    x_train = x_train_resampled
    y_train = y_train_resampled
    
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    #Run logistic regression on training data only to fit
    logreg = LogisticRegression(max_iter=10000)
    logreg.fit(x_train, y_train)

    y_pred = logreg.predict(x_train)
    y_prob = logreg.predict_proba(x_train)[:, 1]

    #Find optimal threshold
    fpr, tpr, thresholds = roc_curve(y_train, y_prob)
    optimal_idx = np.argmax(tpr - fpr) #check if this is right
    optimal_threshold = thresholds[optimal_idx]
    optimal_thresholds_all[fold_idx] = optimal_threshold

    #Predict test fold and apply the optimal decision threshold
    y_test_pred = logreg.predict_proba(x_test)[:, 1]
    y_test_pred_optimal = (y_test_pred >= optimal_threshold).astype(int)



   
